In [ ]:
import numpy as np
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import random
from scipy.io import mmread
import math
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import os
from matplotlib import gridspec
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import chi2,f_classif,mutual_info_classif,mutual_info_regression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from scipy.stats import chisquare
from scipy.stats import pearsonr
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from parfor import parfor
import time
from time import sleep
from parfor import parfor
from scipy import io as sio
from scipy import sparse as ss
from scipy import optimize as so
from sklearn.utils.sparsefuncs import mean_variance_axis
from scipy.stats import linregress
import csv
import seaborn as sns
import copy
from sklearn.cluster import KMeans
import pickle
import numpy as np
import pandas as pd

In [ ]:
def text_create(path, name, msg):
    full_path = path + "/" + name + '.txt'
    file = open(full_path, 'w')
    file.write(str(msg))

def SVM(X, y):
    model = svm.LinearSVC(max_iter=1000000)
    model.fit(X, y)
    return model


def get_error(model, X, y):
    y_pred = model.predict(X)
    return mean_squared_error(y_pred, y)

def feature_compare(path,X, Y, X_test,Y_test,num_features, num_samples_list):
    train_error=[]
    test_error=[]
    train_acc=[]
    test_acc=[]
    D = np.shape(X)[1]
    # Random
    ran_idx = random.choices(range(D), k=num_features)
    text_create(path,'ran_idx',ran_idx)
    for i in range(num_features):
        model=SVM(X[:,ran_idx[:(i+1)]],Y)
        train_error.append(get_error(model,X[:,ran_idx[:(i+1)]],Y))
        train_acc.append(model.score(X[:,ran_idx[:(i+1)]],Y))
        model=SVM(X_test[:,ran_idx[:(i+1)]],Y_test)
        test_error.append(get_error(model,X_test[:,ran_idx[:(i+1)]],Y_test))
        test_acc.append(model.score(X_test[:,ran_idx[:(i+1)]],Y_test))
    print('ran_idx:')
    print(ran_idx)
    
    # correlation coefficient 
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = SelectKBest(f_classif, k=D).fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        score_raw = np.abs(model.scores_).tolist()
        score = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in score_raw]
        idx = model.get_support(indices=True).tolist()
        return [x for _, x in sorted(zip(score, idx), reverse=True)]
    cor_idx=[]
    for i in range(num_features):
        cor_idx.append(tmp[i][:(i+1)])
    text_create(path,'cor_idx',cor_idx)
    for i in range(num_features):
        model=SVM(X[:,cor_idx[i]],Y)
        train_error.append(get_error(model,X[:,cor_idx[i]],Y))
        train_acc.append(model.score(X[:,cor_idx[i]],Y))
        model=SVM(X_test[:,cor_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,cor_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,cor_idx[i]],Y_test))
    print('cor_idx:')
    print(cor_idx)
    
    # mutual
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = SelectKBest(mutual_info_classif, k=D).fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        score_raw = np.abs(model.scores_).tolist()
        score = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in score_raw]
        idx = model.get_support(indices=True).tolist()
        return [x for _, x in sorted(zip(score, idx), reverse=True)]
    mut_idx=[]
    for i in range(num_features):
        mut_idx.append(tmp[i][:(i+1)])
    text_create(path,'mut_idx',mut_idx)
    for i in range(num_features):
        model=SVM(X[:,mut_idx[i]],Y)
        train_error.append(get_error(model,X[:,mut_idx[i]],Y))
        train_acc.append(model.score(X[:,mut_idx[i]],Y))
        model=SVM(X_test[:,mut_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,mut_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,mut_idx[i]],Y_test))
    print('mut_idx:')
    print(mut_idx)
    
    # chi2
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = SelectKBest(chi2, k=D).fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        score_raw = np.abs(model.scores_).tolist()
        score = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in score_raw]
        idx = model.get_support(indices=True).tolist()
        return [x for _, x in sorted(zip(score, idx), reverse=True)]
    chi_idx=[]
    for i in range(num_features):
        chi_idx.append(tmp[i][:(i+1)])
    text_create(path,'chi_idx',chi_idx)
    for i in range(num_features):
        model=SVM(X[:,chi_idx[i]],Y)
        train_error.append(get_error(model,X[:,chi_idx[i]],Y))
        train_acc.append(model.score(X[:,chi_idx[i]],Y))
        model=SVM(X_test[:,chi_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,chi_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,chi_idx[i]],Y_test))
    print('chi_idx:')
    print(chi_idx)
    
    # Feature importance is an inbuilt class that comes with Tree Based Classifiers
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = ExtraTreesClassifier()
        model.fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        coef = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in np.abs(model.feature_importances_)]
        return sorted(range(len(coef)), key=lambda i: coef[i], reverse=True)
    tree_idx=[]
    for i in range(num_features):
        tree_idx.append(tmp[i][:(i+1)])
    text_create(path,'tree_idx',tree_idx)
    for i in range(num_features):
        model=SVM(X[:,tree_idx[i]],Y)
        train_error.append(get_error(model,X[:,tree_idx[i]],Y))
        train_acc.append(model.score(X[:,tree_idx[i]],Y))
        model=SVM(X_test[:,tree_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,tree_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,tree_idx[i]],Y_test))
    print('tree_idx:')
    print(tree_idx)
    
    # SVM by weight
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model=SVM(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        weight=np.sum(np.abs(model.coef_),axis=0).tolist()
        coef = [0 if math.isnan(x) or x==float('inf') else x for x in weight]
        return sorted(range(len(coef)), key=lambda i: coef[i], reverse=True)
    svm_idx=[]
    for i in range(num_features):
        svm_idx.append(tmp[i][:(i+1)])
    text_create(path,'svm_idx',svm_idx)
    for i in range(num_features):
        model=SVM(X[:,svm_idx[i]],Y)
        train_error.append(get_error(model,X[:,svm_idx[i]],Y))
        train_acc.append(model.score(X[:,svm_idx[i]],Y))
        model=SVM(X_test[:,svm_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,svm_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,svm_idx[i]],Y_test))
    print('svm_idx:')
    print(svm_idx)
    text_create(path,'compare_train_error',train_error)
    text_create(path,'compare_test_error',test_error)
    text_create(path,'compare_train_acc',train_acc)
    text_create(path,'compare_test_acc',test_acc)
    return train_error,test_error,train_acc,test_acc,ran_idx,cor_idx,mut_idx,chi_idx,tree_idx,svm_idx



In [ ]:
data_file='./data/MM_all_matrix.mtx'
gene_file='./data/genes.tsv'
label_file='./data/MM_all_metadata.csv'

M, raw_gene, meta = load_consolidated_data(data_file, label_file, gene_file)
M2, meta = normalization(M, meta)
meta = pd.read_csv(label_file,header=0)

l=[]
for i in range(np.shape(raw_gene)[0]):
    if raw_gene[i][:3]!='RPL' and raw_gene[i][:3]!='RPS' and raw_gene[i][:2]!='MT':
        l.append(i)
gene=raw_gene[l]

data=normalize(M2.tocsr().transpose()[:,l],axis=1, norm='l2')

raw_target=meta["disease"].values
target=(raw_target=='MM')*1.0
target=target.astype(np.uint8)
classes=range(len(np.unique(target)))
keys=['healthy','MM']
del M,M2,raw_gene,meta,raw_target

idx = np.arange(np.shape(data)[0])
random.shuffle(idx)
X_train = data[idx[:int(np.shape(data)[0]*4/5)],:]
y_train = target[idx[:int(np.shape(data)[0]*4/5)]]
X_test = data[idx[int(np.shape(data)[0]*4/5):],:]
y_test = target[idx[int(np.shape(data)[0]*4/5):]]

print(type(data))
print(np.shape(data),np.shape(target),len(np.unique(target)))
print(np.shape(X_train))
print(np.shape(X_test))
for i in np.unique(target):
    print('class '+keys[i]+': '+str(np.count_nonzero((target==i)*1)))

In [ ]:
'''
Parameters
----------
balance : boolean
    balance the number of cells of each class or just randomly select cells at each loop
num_features : int
    the total number of genes we want to select 
num_samples : int
    the number of cells we would use at each loop
'''
num_features = 500
num_samples_list=[100, 185, 200, 220, 243, 265, 279, 291, 302, 315, 319, 319, 330, 330, 330, 340, 377, 387, 389, 392, 400, 400, 406, 408, 408, 408, 412, 412, 417, 422, 423, 423, 427, 428, 428, 428, 428, 428, 428, 431, 433, 433, 436, 436, 440, 442, 444, 446, 450, 450, 450, 450, 454, 456, 457, 457, 457, 458, 464, 467, 467, 468, 468, 471, 471, 471, 472, 472, 472, 472, 472, 472, 472, 476, 477, 477, 477, 477, 477, 477, 479, 479, 480, 481, 482, 483, 483, 483, 483, 483, 483, 486, 486, 487, 487, 488, 488, 490, 490, 490, 490, 490, 490, 490, 490, 491, 491, 491, 492, 492, 492, 492, 493, 493, 494, 495, 496, 496, 497, 497, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 498, 499, 499, 500, 500, 500, 501]

path='results/compare_mincell'
try:
    os.mkdir('results')
except OSError:
    print ("Creation of the directory %s failed")
else:
    print ("Successfully created the directory %s ")
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

train_error,test_error,train_acc,train_acc,ran_idx,fvalue_idx,mut_idx,chi_idx,tree_idx,svm_idx=feature_compare(path,X_train, y_train, X_test,y_test,num_features, num_samples_list)

